## Download 지속가능경영보고서 files

In [15]:
# 웹 table 크롤링에 필요한 패키지
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions

# 오류(max url 초과) 해결을 위한 패키지
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

#기본 패키지들
import os
import pyperclip
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# For macro rest
import time
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

# pdf 다운로드 창 option 변경 
from selenium.webdriver.chrome.options import Options


import warnings
warnings.filterwarnings(action='ignore')

#change working dir.
os.chdir('E:/루당탕사라')

## KSA 한국표준협회 ESG 경영추진단에서 지가경 다운받기
https://www.ksaesg.or.kr/p_base.php?action=h_report_04&s_text=2022%EB%85%84&s_category=&page=&1=1  

2022년으로 검색. PAGE 총 5페이지.  
검색과 page index는 모두 url에서 변경 가능 (따라서 page 이동은 url 수정을 통한 for문으로 돌릴 것.)

In [10]:
# data 넣을 DF 정의 ------------------------------------------------------------------------

SR_df = pd.DataFrame({'기업명':[], '발간시기':[], '산업구분':[], '보고서명':[]})



# 크롤링 시작 -------------------------------------------------------------------------------

path = "chromedriver.exe"

wd = webdriver.Chrome(path)
wd.maximize_window() #창 최대화

# 사이트로 접속
for page in range(5):
    
    # page 입력해서 url 정의
    url = 'https://www.ksaesg.or.kr/p_base.php?action=h_report_04&board_id=&s_text=2022%EB%85%84&s_category=&page=' + str(page+1)
    
    # url로 이동
    wd.get(url)
    time.sleep(3)
    
    print(str(page+1) + " page is crawled...")
    
    # 보고서명 → 지가경 다운로드
    ## pdf 수 추출
    pdf_num = len(wd.find_elements_by_xpath('/html/body/div[1]/div[4]/div/div/div[1]/div[2]/div/div/table/tbody/tr/td[4]/a'))
    time.sleep(1)
    
    ## pdf 수만큼 for문 돌리기
    for num in range(pdf_num):
        ### pdf link 추출
        pdf_link = wd.find_element_by_xpath(f'/html/body/div[1]/div[4]/div/div/div[1]/div[2]/div/div/table/tbody/tr[{num+1}]/td[4]/a')
        time.sleep(1)
        
        ### link로 접속
        pdf_link.click()
        time.sleep(2)
        
        ### 보고서명 클릭, pdf 다운
        wd.find_element_by_xpath('//*[@class="btn btn-default c-btn-uppercase c-btn-bold"]').click()
        wd.implicitly_wait(3)
        
        wd.back() # 뒤로가기
        time.sleep(3)

    
    # table 내용 추출
    elements = []
    elements.append(wd.find_elements_by_xpath('/html/body/div[1]/div[4]/div/div/div[1]/div[2]/div/div/table/tbody/tr/th'))
    elements.append(wd.find_elements_by_xpath('/html/body/div[1]/div[4]/div/div/div[1]/div[2]/div/div/table/tbody/tr/td[2]'))
    elements.append(wd.find_elements_by_xpath('/html/body/div[1]/div[4]/div/div/div[1]/div[2]/div/div/table/tbody/tr/td[3]'))
    elements.append(wd.find_elements_by_xpath('/html/body/div[1]/div[4]/div/div/div[1]/div[2]/div/div/table/tbody/tr/td[4]/a'))
    
    # table 내용에서 text 추출
    text = []
    for i, ele in enumerate(elements):
        text.append([x.text for x in ele])
    
    # new_df 생성 & concat
    new_df = pd.DataFrame({'기업명':text[0], '발간시기':text[1], '산업구분':text[2], '보고서명':text[3]})
    SR_df = pd.concat([SR_df, new_df])

    
    # 다음 페이지로 넘어가기 전에 5초 휴식
    time.sleep(5)


print('크롤링 종료.')
# 다 다운받을 수 있게끔 wd.quit()는 생략.

1 page is crawled...
2 page is crawled...
3 page is crawled...
4 page is crawled...
5 page is crawled...
크롤링 종료.


In [11]:
SR_df.head(20)

,기업명,발간시기,산업구분,보고서명
0,한국산림복지진흥원,2022년 8월,기타,2021-2022 한국산림복지진흥원 지속가능경영보고서
1,LX인터내셔널,2022년 7월,서비스,LX인터내셔널 2022 지속가능경영 보고서
2,대신증권,2022년 8월,서비스 > 증권,2022 대신증권 통합보고서
3,효성ITX,2022년 7월,서비스,효성ITX 2022 지속가능경영보고서
4,환경재단,2022년 4월,기타,환경재단 2021 지속가능보고서
5,화신,2022년 5월,제조 > 자동차부품,2022 화신 지속가능경영보고서
6,현대일렉트릭,2022년 6월,제조,현대일렉트릭 통합보고서 2021
7,현대건설기계,2022년 6월,제조 > 선업용기계,2021 현대건설기계 통합보고서
8,한화에어로스페이스,2022년 8월,제조,한화에어로스페이스 지속가능경영보고서 2022
9,한화,2022년 6월,제조,한화 지속가능경영보고서 2022


In [16]:
SR_df.to_csv('지가경/SR_list.csv', index=None, encoding='cp949')

## [NOTE]

결과를 보니 221개의 PDF가 다운받아졌다.  
나머지 4개의 기업은 한화, 한화시스템, 한화에어로스페이스, 현대제철로,  
페이지로 접근했을 때 연결할 수 없다는 문구가 뜨며 pdf를 다운받지 못했다.

따라서 직접 기업명을 검색해서 지가경 pdf를 다운받았다.

## [NOTE 2]

다운로드 받은 지속가능경영보고서 PDF는 ZIP파일로 저장하여 지가경 폴더에 넣었다.  
다운받은 PDF는 221개에 4개를 추가한 225개이다.  
그러나 아예 SR_list에 없었으나 추가적으로 PDF를 찾은 경우 / list에 명단이 있으나 ESG 정보는 다른 PDF가 있는 경우: PDF를 다운받아 폴더에 추가하였다.